In [1]:
import numpy as np
from sympy import Matrix, Symbol
import sympy

        
class CplxMatrix:
    def __init__(self,name, lines, columns):
        self.name = name
        self.lines = lines
        self.columns = columns
    
    def flat_index(self,line,column):
        assert line < self.lines
        assert column < self.columns
        index = 2*(column + line*self.columns)
        return index
    
    def ccode_item_re(self, line,column):
        return f"{self.name}[{self.flat_index(line, column)}]"
    
    def ccode_item_im(self, line,column):
        index = self.flat_index(line, column)+1
        return f"{self.name}[{index}]"
        
        
class RealMatrix:
    def __init__(self,name,lines, columns):
        self.name = name
        self.lines = lines
        self.columns = columns
    
    def flat_index(self,line,column):
        assert line < self.lines
        assert column < self.columns
        index = 2*(column + line*self.columns)
        return index
    
    def ccode_item_re(self, line,column):
        return f"{self.name}[{self.flat_index(line, column)}]"
    
    def ccode_item_im(self, line,column):
        return f"0"
    

class CplxTriangularMatrix:
    def __init__(self,name,lines, columns):
        self.name = name
        self.lines = lines
        self.columns = columns
    
    def flat_index(self,line,column):
        if line > column:
            return self.flat_index(column, line)
        else:
            index = (2*(column + line*self.columns)) - 2*int(((line+.5)**2)/2) - line -(1 * column>line)
            return index
    
    def ccode_item_re(self, line,column):
        return f"{self.name}[{self.flat_index(line, column)}]"
    
    def ccode_item_im(self, line,column):
        index = self.flat_index(line, column)+1
        if line > column:
            return f"(-{self.name}[{index}])"
        elif line < column:
            return f"{self.name}[{index}]"
        return f"0"


class ASM_HW_order_CplxTriangularMatrix:
    def __init__(self,name,lines, columns, frequencies):
        self.name = name
        self.lines = lines
        self.columns = columns
        self.frequencies = frequencies
    
    def flat_index(self,line,column):
        if line > column:
            return self.flat_index(column, line)
        else:
            index = (2*(column + line*self.columns)) - 2*int(((line+.5)**2)/2) - line -(1 * column>line)
            return index * self.frequencies
    
    def ccode_item_re(self, line,column):
        return f"{self.name}[{self.flat_index(line, column)}]"
    
    def ccode_item_im(self, line,column):
        index = self.flat_index(line, column)+1
        if line > column:
            return f"(-{self.name}[{index}])"
        elif line < column:
            return f"{self.name}[{index}]"
        return f"0"


class ASM_SW_order_CplxTriangularMatrix:
    def __init__(self,name,lines, columns, frequencies):
        self.name = name
        self.lines = lines
        self.columns = columns
    
    def flat_index(self,line,column):
        if line > column:
            return self.flat_index(column, line)
        else:
            index = (2*(column + line*self.columns)) - 2*int(((line+.5)**2)/2) - line -(1 * column>line)
            return index
    
    def ccode_item_re(self, line,column):
        return f"{self.name}[{self.flat_index(line, column)}]"
    
    def ccode_item_im(self, line,column):
        index = self.flat_index(line, column)+1
        if line > column:
            return f"(-{self.name}[{index}])"
        elif line < column:
            return f"{self.name}[{index}]"
        return f"0"

In [2]:
from sympy import Matrix, Symbol
B1 = Symbol("B1", complex=True, commutative = True)
B2 = Symbol("B2", complex=True, commutative = True)
B3 = Symbol("B3", complex=True, commutative = True)
E1 = Symbol("E1", complex=True, commutative = True)
E2 = Symbol("E2", complex=True, commutative = True)

B = Matrix([B1, B2, B3, E1, E2])

M = B * B.conjugate().transpose()
M

Matrix([
[B1*conjugate(B1), B1*conjugate(B2), B1*conjugate(B3), B1*conjugate(E1), B1*conjugate(E2)],
[B2*conjugate(B1), B2*conjugate(B2), B2*conjugate(B3), B2*conjugate(E1), B2*conjugate(E2)],
[B3*conjugate(B1), B3*conjugate(B2), B3*conjugate(B3), B3*conjugate(E1), B3*conjugate(E2)],
[E1*conjugate(B1), E1*conjugate(B2), E1*conjugate(B3), E1*conjugate(E1), E1*conjugate(E2)],
[E2*conjugate(B1), E2*conjugate(B2), E2*conjugate(B3), E2*conjugate(E1), E2*conjugate(E2)]])

In [3]:
from sympy import Matrix, Symbol
B1B1 = Symbol("B1B1'", real=True, commutative = True)
B1B2 = Symbol("B1B2'", complex=True, commutative = True)
B1B3 = Symbol("B1B3'", complex=True, commutative = True)
B1E1 = Symbol("B1E1'", complex=True, commutative = True)
B1E2 = Symbol("B1E2'", complex=True, commutative = True)

B2B1 = Symbol("B2B1'", complex=True, commutative = True)
B2B2 = Symbol("B2B2'", real=True, commutative = True)
B2B3 = Symbol("B2B3'", complex=True, commutative = True)
B2E1 = Symbol("B2E1'", complex=True, commutative = True)
B2E2 = Symbol("B2E2'", complex=True, commutative = True)

B3B1 = Symbol("B3B1'", complex=True, commutative = True)
B3B2 = Symbol("B3B2'", complex=True, commutative = True)
B3B3 = Symbol("B3B3'", real=True, commutative = True)
B3E1 = Symbol("B3E1'", complex=True, commutative = True)
B3E2 = Symbol("B3E2'", complex=True, commutative = True)

E1B1 = Symbol("E1B1'", complex=True, commutative = True)
E1B2 = Symbol("E1B2'", complex=True, commutative = True)
E1B3 = Symbol("E1B3'", complex=True, commutative = True)
E1E1 = Symbol("E1E1'", real=True, commutative = True)
E1E2 = Symbol("E1E2'", complex=True, commutative = True)

E2B1 = Symbol("E2B1'", complex=True, commutative = True)
E2B2 = Symbol("E2B2'", complex=True, commutative = True)
E2B3 = Symbol("E2B3'", complex=True, commutative = True)
E2E1 = Symbol("E2E1'", complex=True, commutative = True)
E2E2 = Symbol("E2E2'", real=True, commutative = True)

ASM = Matrix(
    [
        [B1B1,B1B2,B1B3,B1E1,B1E2],
        [B2B1,B2B2,B2B3,B2E1,B2E2],
        [B3B1,B3B2,B3B3,B3E1,B3E2],
        [E1B1,E1B2,E1B3,E1E1,E1E2],
        [E2B1,E2B2,E2B3,E2E1,E2E2]
    ])

ASM

Matrix([
[B1B1', B1B2', B1B3', B1E1', B1E2'],
[B2B1', B2B2', B2B3', B2E1', B2E2'],
[B3B1', B3B2', B3B3', B3E1', B3E2'],
[E1B1', E1B2', E1B3', E1E1', E1E2'],
[E2B1', E2B2', E2B3', E2E1', E2E2']])

In [4]:
CAL = Matrix([
    [Symbol("C11", complex=True, commutative = True), Symbol("C12", complex=True, commutative = True), Symbol("C13", complex=True, commutative = True), 0, 0],
    [Symbol("C21", complex=True, commutative = True), Symbol("C22", complex=True, commutative = True), Symbol("C23", complex=True, commutative = True), 0, 0],
    [Symbol("C31", complex=True, commutative = True), Symbol("C32", complex=True, commutative = True), Symbol("C33", complex=True, commutative = True), 0, 0],
    [0, 0, 0, Symbol("C44", complex=True, commutative = True), Symbol("C45", complex=True, commutative = True)],
    [0, 0, 0, Symbol("C54", complex=True, commutative = True), Symbol("C55", complex=True, commutative = True)]
])
CAL

Matrix([
[C11, C12, C13,   0,   0],
[C21, C22, C23,   0,   0],
[C31, C32, C33,   0,   0],
[  0,   0,   0, C44, C45],
[  0,   0,   0, C54, C55]])

In [5]:
CAL[0,3]==0

True

In [6]:
R = CAL * M * CAL.transpose().conjugate()

In [7]:
R[0,0]

(B1*C11*conjugate(B1) + B2*C12*conjugate(B1) + B3*C13*conjugate(B1))*conjugate(C11) + (B1*C11*conjugate(B2) + B2*C12*conjugate(B2) + B3*C13*conjugate(B2))*conjugate(C12) + (B1*C11*conjugate(B3) + B2*C12*conjugate(B3) + B3*C13*conjugate(B3))*conjugate(C13)

In [8]:
sympy.im(R[0,0])

-(re(C11)*Abs(B1)**2 + re(B2*C12*conjugate(B1)) + re(B3*C13*conjugate(B1)))*im(C11) - (re(C12)*Abs(B2)**2 + re(B1*C11*conjugate(B2)) + re(B3*C13*conjugate(B2)))*im(C12) - (re(C13)*Abs(B3)**2 + re(B1*C11*conjugate(B3)) + re(B2*C12*conjugate(B3)))*im(C13) + (im(C11)*Abs(B1)**2 + im(B2*C12*conjugate(B1)) + im(B3*C13*conjugate(B1)))*re(C11) + (im(C12)*Abs(B2)**2 + im(B1*C11*conjugate(B2)) + im(B3*C13*conjugate(B2)))*re(C12) + (im(C13)*Abs(B3)**2 + im(B1*C11*conjugate(B3)) + im(B2*C12*conjugate(B3)))*re(C13)

In [9]:
sympy.simplify(sympy.im(sympy.expand_complex(R[0,0])))

0

In [10]:
ASM[0,0]

B1B1'

In [11]:
import re

input_matrix = CplxTriangularMatrix("input_asm",*ASM.shape)
calibrated_matrix = CplxTriangularMatrix("output_asm",*ASM.shape)

replace_rules = {}
for line in range(ASM.shape[0]):
    for column in range(ASM.shape[1]):
        if line == column:
            replace_rules[f"{ASM[line, column]}"] = input_matrix.ccode_item_re(line, column)
        else:
            replace_rules[f"re({ASM[line, column]})"] = input_matrix.ccode_item_re(line, column)
            replace_rules[f"im({ASM[line, column]})"] = input_matrix.ccode_item_im(line, column)
            
IM = Matrix([[Symbol(f"IM{line}{column}", complex=True, commutative = True) for column in range(5)] for line in range(5)])

intermediary_matrix = CplxMatrix("intermediary",*ASM.shape)            
for line in range(IM.shape[0]):
    for column in range(IM.shape[1]):
        replace_rules[f"re({IM[line, column]})"] = intermediary_matrix.ccode_item_re(line, column)
        replace_rules[f"im({IM[line, column]})"] = intermediary_matrix.ccode_item_im(line, column)
        
        
BCAL_matrix = CplxMatrix("mag_calibration_matrices",3,3)
ECAL_matrix = CplxMatrix("elec_calibration_matrices",2,2)
for line in range(CAL.shape[0]):
    for column in range(CAL.shape[1]):
        if CAL[line, column] != 0:
            if column < 3:
                replace_rules[f"re({CAL[line, column]})"] = BCAL_matrix.ccode_item_re(line, column)
                replace_rules[f"im({CAL[line, column]})"] = BCAL_matrix.ccode_item_im(line, column)
            else:
                replace_rules[f"re({CAL[line, column]})"] = ECAL_matrix.ccode_item_re(line-3, column-3)
                replace_rules[f"im({CAL[line, column]})"] = ECAL_matrix.ccode_item_im(line-3, column-3)

replace_rules = dict((re.escape(k), v) for k, v in replace_rules.items()) 
pattern = re.compile("|".join(replace_rules.keys()))


In [12]:
II = CAL * ASM
Calibrated_Matrix = IM * CAL.transpose().conjugate()

In [13]:

def simplify_im(expr):
    return sympy.simplify(sympy.im(sympy.expand_complex(expr)))

def simplify_re(expr):
    return sympy.simplify(sympy.re(sympy.expand_complex(expr)))

In [14]:
code = ''

for line in range(II.shape[0]):
    for column in range(II.shape[1]):
        code += f"{intermediary_matrix.ccode_item_re(line, column)} = " + pattern.sub(lambda m: replace_rules[re.escape(m.group(0))], 
                           str(simplify_re(II[line,column]))) + ";\n"
        
        code += f"{intermediary_matrix.ccode_item_im(line, column)} = " + pattern.sub(lambda m: replace_rules[re.escape(m.group(0))], 
                           str(simplify_im(II[line,column]))) + ";\n"
        
for line in range(Calibrated_Matrix.shape[0]):
    for column in range(Calibrated_Matrix.shape[1]):
        if line <= column:
            code += f"{calibrated_matrix.ccode_item_re(line, column)} = " + pattern.sub(lambda m: replace_rules[re.escape(m.group(0))], 
                               str(simplify_re(Calibrated_Matrix[line,column]))) + ";\n"
        if column > line:
            code += f"{calibrated_matrix.ccode_item_im(line, column)} = " + pattern.sub(lambda m: replace_rules[re.escape(m.group(0))], 
                               str(simplify_im(Calibrated_Matrix[line,column]))) + ";\n"
            
        

In [15]:
print(code)

intermediary[0] = input_asm[0]*mag_calibration_matrices[0] + input_asm[1]*mag_calibration_matrices[2] + input_asm[3]*mag_calibration_matrices[4] - (-input_asm[2])*mag_calibration_matrices[3] - (-input_asm[4])*mag_calibration_matrices[5];
intermediary[1] = input_asm[0]*mag_calibration_matrices[1] + input_asm[1]*mag_calibration_matrices[3] + input_asm[3]*mag_calibration_matrices[5] + mag_calibration_matrices[2]*(-input_asm[2]) + mag_calibration_matrices[4]*(-input_asm[4]);
intermediary[2] = input_asm[9]*mag_calibration_matrices[2] + input_asm[1]*mag_calibration_matrices[0] + input_asm[10]*mag_calibration_matrices[4] - input_asm[2]*mag_calibration_matrices[1] - (-input_asm[11])*mag_calibration_matrices[5];
intermediary[3] = input_asm[9]*mag_calibration_matrices[3] + input_asm[1]*mag_calibration_matrices[1] + input_asm[10]*mag_calibration_matrices[5] + mag_calibration_matrices[0]*input_asm[2] + mag_calibration_matrices[4]*(-input_asm[11]);
intermediary[4] = input_asm[16]*mag_calibration_ma